In [1]:
import torch
import numpy as np

# Introduccion a Pytorch

Pytorch es una libreria que permite trabajar con vectores y matrices de muchas dimensiones.

En ese sentido es muy parecido a Numpy. En numpy a los vectores son llamados arrays. En pytorch se los llama tensores.

In [2]:
a = torch.tensor([[1.,2.],[3.,4.]])
a

tensor([[1., 2.],
        [3., 4.]])

In [3]:
a.shape

torch.Size([2, 2])

El shape del tensor es una lista del tamano de cada dimension. Si trabajamos con matrices (como comumnmente se llama a los vectores 2D), cada "fila" tiene la misma longitud: no hay una fila mas corta que otra.

Lo mismo vale para mas dimensiones. No podemos hacer:

In [4]:
torch.tensor([[1.,2.],[3.]])

ValueError: expected sequence of length 2 at dim 1 (got 1)

## Que introduce de nuevo pytorch sobre numpy
Esencialmente dos cosas:
- Paralelizacion en GPU
- Calculo automatico de derivadas (gradiente)

Estas funciones son muy deseables cuando trabajamos con redes neuronales. Aunque tambien hay librerias de, por ejemplo, algebra lineal que han aprovechado esto para ser escritas sobre pytorch y funcionar eficientemente.

Pytorch tambien provee muchas de las funcionalidades necesarias para definir una red y entrenarla.

Con la siguiente funcion podemos ver si tenemos una GPU disponible en nuestro sistema

In [5]:
torch.cuda.is_available()

False

Podemos comparar la velocidad entre ejecutar algo en CPU o usando GPU (si tenemos).

Pytorch requiere mover explicitamente los tensores a la GPU, se puede hacer facilmente mediante el metodo .cuda()

(Si se operan vectores que estan en la GPU con vectores que estan en la GPU causara un error)

In [10]:
A = torch.rand(1000,1000)
B = torch.rand(1000,1000)

In [11]:
%timeit A@B

1.54 ms ± 41.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
if torch.cuda.is_available():
    A = A.cuda()
    B = B.cuda()
    print("CUDA available")
else:
    print("CUDA not available")

CUDA not available


In [13]:
%timeit A@B

1.53 ms ± 17 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Si tenés iOS (Mac) podés ver como acelerar Pytorch con el backedn MPS (solo disponible para AMD o Sillicon Chip, NO Intel).
[Ver la documentación](https://developer.apple.com/metal/pytorch/)

In [14]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("MPS available")
    A = A.to(mps_device)
    B = B.to(mps_device)
else:
    print ("MPS device not found.")

MPS available


In [15]:
%timeit A@B

721 μs ± 4.06 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Ejercicio 1: "Usando una GPU" (opcional)

Ejecutar el codigo anterior en una GPU. Para ello se pueden utilizar las GPU de papparspace u otro proveedor cloud.

## Pytorch Autograd

Pytorch provee la funcionalidad "recordar" como cada vector fue calculado con el fin de computar el gradiente.

Podemos definir vectores con ciertos valores y crear otros como resultado de operar los primeros y pytorch recordara el grafo de las computaciones.

Supongamos que tenemos los valores:

$ a = 1 $

$ b = 2 $

$ c = 0 $

Y definimos $m$, $n$ (capas intermedias), $p$ (una prediccion) y $l$ (un costo) como:

$ m = a + b $

$ n = max(b,c) $

$ p = m \times n $

$ l = p^2 $

Pytorch calculara los valores intermedios dado el valor de las hojas:

$ m = 3 $

$ n = 2 $

$ p = 6 $

$ l = 36 $

Y a la vez construira el siguiente grafo de computaciones:

![title](graph.png)

Si $a$,$b$,$c$ son nuestros parametros y queremos minimizar el costo $l$. Querremos calcular: $\Large\frac{\partial l}{\partial a}$, $\Large\frac{\partial l}{\partial b}$, $\Large\frac{\partial l}{\partial c}$

A primera vista no es fácil calcular dichas derivadas. En general, usando la definición de los valores, podemos calcular la derivada de un valor en función de los valores que dependen de forma directa de él. Haciendo referencia al grafo de las computaciones (la imagen de arriba), podemos calcular el valor de $\large\frac{\partial y}{\partial x}$ si existe una arista que $x \rightarrow y$. Así, podemos calcular las siguientes derivadas, ya que cada variable depende de forma directa una de la otra:

$\large\frac{\partial l}{\partial p} = 2 \times p = 12$

$\large\frac{\partial p}{\partial m} = n = 2$

$\large\frac{\partial p}{\partial n} = m = 3$

$\large\frac{\partial m}{\partial a} = 1$

$\large \frac{\partial m}{\partial b} = 1$

$\large \frac{\partial n}{\partial b} = 1$

$\large \frac{\partial n}{\partial c} = 0$

Pero como hacemos para calcular las derivadas $\Large\frac{\partial l}{\partial a}$, $\Large\frac{\partial l}{\partial b}$, $\Large\frac{\partial l}{\partial c}$ que son las que necesitamos para actualizar el gradiente? 

Centrémosnos por un momento en el caso de $\Large\frac{\partial l}{\partial a}$. Si bien no tenemos una fórmula que relacione de manera directa $a$ con $l$, el valor de $l$ depende del valor de $a$: esto es debido a que el valor de $l$ depende de $p$, que a su vez depende de $m$, que por último depende de $a$.

Apliquemos la [regla de la cadena](https://en.wikipedia.org/wiki/Chain_rule) entre $p$ (que es una función de $a$), $l$ (que es función de $p$) y $a$:

$\Large\frac{\partial l}{\partial a} = \frac{\partial l}{\partial p} \times \frac{\partial p}{\partial a}$

Mirando el término de la derecha:
- $\Large \frac{\partial l}{\partial p}$ lo conocemos y vale $12$.
- $\Large \frac{\partial p}{\partial a}$ no lo hemos calculado aún, pero nos hemos "acercado" a $a$ en el grafo de las computaciones. Todo indica que si aplicamos una vez más la regla de la cadena terminaremos llegando a $a$. Apliquemos pues, la regla de la cadena sobre $m$, $p$ y $a$:

$\Large\frac{\partial p}{\partial a} = \frac{\partial p}{\partial m} \times \frac{\partial m}{\partial a}$

Ahora, mirando el término de la derecha, tenemos todos los valores: $\Large \frac{\partial p}{\partial m}$ vale 2 y $\Large \frac{\partial m}{\partial a}$ vale 1.

Ya estamos en condiciones pues de calcular $\Large\frac{\partial l}{\partial a}$:

$\Large\frac{\partial l}{\partial a} = \frac{\partial l}{\partial p} \times \frac{\partial p}{\partial a} = 12 \times \frac{\partial p}{\partial m} \times \frac{\partial m}{\partial a} = 12 \times 2 \times 1 = 24$

Ordenando nuestro razonamiento, lo que estamos haciendo es calcular los gradientes respecto de las variables que están más cerca del resultado:

$\Large\frac{\partial l}{\partial m} = \frac{\partial l}{\partial p} \times \frac{\partial p}{\partial m} = 12 \times 2 = 24$



$\Large\frac{\partial l}{\partial n} = \frac{\partial l}{\partial p} \times \frac{\partial p}{\partial n} = 12 \times 3 = 36$

Y luego utilizar dichos gradientes para calcular los de las capas anteriores:

$\Large\frac{\partial l}{\partial a} = \frac{\partial l}{\partial m} \times \frac{\partial m}{\partial a} = 24 \times 1 = 24$


Podemos hacer lo mismo para calcular $\Large\frac{\partial l}{\partial c}$.


Para el caso de $b$ usamos la [regla de la cadena multivariada](https://math.hmc.edu/calculus/hmc-mathematics-calculus-online-tutorials/multivariable-calculus/multi-variable-chain-rule/) (es muy parecida a la regla del producto, de hecho la regla del producto es un caso particular de la regla de la cadena multivariada). Esta aplica porque $b$ es usada en varios valores ($m$ y $n$) de los cuales depende en última instancia el valor que queremos diferenciar $l$.

$\Large\frac{\partial l}{\partial b} = \frac{\partial l}{\partial m} \times \frac{\partial m}{\partial b} + \frac{\partial l}{\partial n} \times \frac{\partial n}{\partial b} = 24+36 = 60$

Ahora vamos a calcular los gradientes usando [Pytorch Autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) (diferenciación automática).

Primero definimos los valores hoja (de los que depende el resto). Serían nuestro parámetros (respecto de los cuales vamos a derivar después). Es importante el argumento requires_grad para que Pytorch sepa que tiene que calcular el grafo de las computaciones que le apliquemos a dichas variables, porque vamos a querer el gradiente respecto a dichas variables:

In [16]:
a = torch.tensor([1.],requires_grad=True)
b = torch.tensor([2.],requires_grad=True)
c = torch.tensor([0.],requires_grad=True)
a,b,c

(tensor([1.], requires_grad=True),
 tensor([2.], requires_grad=True),
 tensor([0.], requires_grad=True))

Ahora podemos definir otros valores en relación a los anteriores. En grad_fn lo que vemos es que Pytorch está llevando un historial de como dichos valores fueron contruídos. Esto es el grafo de las computaciones que luego le servirá para calcular el gradiente:

In [19]:
m = a+b
n = torch.max(a,b)
m, n

(tensor([3.], grad_fn=<AddBackward0>),
 tensor([2.], grad_fn=<MaximumBackward0>))

In [21]:
p = m*n
l = p**2
p, l

(tensor([6.], grad_fn=<MulBackward0>), tensor([36.], grad_fn=<PowBackward0>))

Para calcular el gradiente de $l$ basta con llamar l.backwards()

Generalmente sólo usamos los gradientes respecto a los parámetros (las hojas). Si bien Pytorch calcula los gradientes respecto a los nodos intermedios (es un resultado parcial que luego ayudará para calcular los gradientes en las hojas), los borra automáticamente luego de usarlos para ahorrar memoria. Esto se puede evitar con el método .retain_grad(). En este caso le pediremos a Pytorch que no los borre ya que los queremos mostrar para corroborar que nos dio el mismo resultado que el que hallamos de manera analítica:

In [22]:
m.retain_grad()
n.retain_grad()
p.retain_grad()
l.retain_grad()

l.backward()

Por último mostramos los gradientes de $l$ respecto a cada una de las variables. Podemos corroborar que los valores son los mismo que los hallados de manera analítica arriba:

In [23]:
l.grad, p.grad

(tensor([1.]), tensor([12.]))

In [24]:
m.grad, n.grad

(tensor([24.]), tensor([36.]))

In [25]:
a.grad, b.grad, c.grad

(tensor([24.]), tensor([60.]), None)

### Interpretación del gradiente

Recordar que el gradiente $\Large \frac{\partial l}{\partial a}$ indica cuánto cambia $l$ ante pequeños cambios de $a$:

$\Large \frac{\partial l}{\partial a} = lim_{\Delta a \rightarrow 0} \frac{\Delta l}{\Delta a}$

Corroboraremos esto empíricamente, calculando $\Delta l$ para variaciones $\Delta a$, $\Delta b$, $\Delta c$ muy pequeñas.

Para ellos haremos una función que nos calcule $l$ dado $a$, $b$ y $c$c

In [26]:
def calculate_l(a,b,c):
    m = a+b
    n = torch.max(a,b)
    p = m*n
    l = p**2
    return l

Chequeamos que el resultado es efectivamente 36:

In [27]:
ov = calculate_l(a, b, c)
ov

tensor([36.], grad_fn=<PowBackward0>)

Luego calcularemos $l$ introduciendo un pequeño cambio en $a$. Observaremos como resulta el ratio de cambio $\Large \frac{\Delta l}{\Delta a}$

In [28]:
small_change = 0.001
nv = calculate_l(a + small_change, b, c)
print(f'New value: {(nv).item()}')
print(f'Change value: {(nv - ov).item()}')
print(f'Change ration: {((nv - ov) / small_change).item()}')

New value: 36.02400207519531
Change value: 0.0240020751953125
Change ration: 24.002073287963867


Efectivamente el ratio de variación es 24, como la derivada $\Large \frac{\partial l}{\partial a}$.
Haremos lo mismo para $b$ y para $c$:

In [29]:
nv = calculate_l(a , b + small_change, c)
nv, (nv - ov) / small_change
print(f'New value: {(nv).item()}')
print(f'Change value: {(nv - ov).item()}')
print(f'Change ration: {((nv - ov) / small_change).item()}')

New value: 36.06003189086914
Change value: 0.060031890869140625
Change ration: 60.03188705444336


In [30]:
nv = calculate_l(a , b, c + small_change)
nv, (nv - ov) / small_change
print(f'New value: {(nv).item()}')
print(f'Change value: {(nv - ov).item()}')
print(f'Change ration: {((nv - ov) / small_change).item()}')

New value: 36.0
Change value: 0.0
Change ration: 0.0


#### Ejercicio 2: "Calculo del gradiente" (opcional)

Dada las siguientes definiciones:

$a = 2$

$b = 3$

$c = -1$

$d = 7$

$m = 3 \times a + 4 \times b - 2 \times c + 10 \times d$

$n = 7 \times a + b + \times c + 4 \times d$

$p = max(m,0)$

$q = max(n,0)$

$z = p - q$

$ pred = \frac{1}{1 + e^{-z}}$

$loss = log(pred)$

Calcular:

1. El grafo de las computaciones (un diagrama de flechas como el de arriba donde se vea la relación de dependencia entre las variables).
2. Las derivadas $\large \frac{\partial loss}{\partial a}$, $\large \frac{\partial loss}{\partial b}$, $\large \frac{\partial loss}{\partial c}$, $\large \frac{\partial loss}{\partial d}$ manualmente.
3. Las derivadas $\large \frac{\partial loss}{\partial a}$, $\large \frac{\partial loss}{\partial b}$, $\large \frac{\partial loss}{\partial c}$, $\large \frac{\partial loss}{\partial d}$ utilizando Pytorch Autograd (backwards).
4. Una aproximación de las derivadas $\large \frac{\partial loss}{\partial a}$, $\large \frac{\partial loss}{\partial b}$, $\large \frac{\partial loss}{\partial c}$, $\large \frac{\partial loss}{\partial d}$ mediante los ratios $\large \frac{\Delta loss}{\Delta a}$, $\large \frac{\Delta loss}{\Delta b}$, $\large \frac{\Delta loss}{\Delta c}$, $\large \frac{\Delta loss}{\Delta d}$ de cambio utilizando pequeñas variaciones $\Delta a$, $\Delta b$, $\Delta c$, $\Delta d$.

## Utilizando descenso por gradiente para problemas de optimización

El método de descenso por gradiente permite optimizar (maximizar o minimizar) cualquier función $f(w)$. Consiste comenzar en cualquier $w_0$ aleatorio y hacer iterativamente (una y otra vez) la siguiente actualización:

$w_{t+1} = w_{t} - \eta \times f'(w_t)$

Por ejemplo si queremos hallar el mínimo de la siguiente función cuadrática:

$f(x) = (x-3)^2 + 10$

Podemos hacerlo analíticamente: observar que es una parábola con vértice en $x=3$ y que este es el $x$ que arrojará valores mínimos. Particularmente:

$f(3) = 10$


Pero también podemos calcular el mínimo con el método de descenso por gradiente:

In [31]:
def f(x):
    return (x-3)**2+10

In [32]:
w = torch.rand(1, requires_grad=True)
w

tensor([0.9700], requires_grad=True)

La siguiente celda ejecuta un paso de descenso por gradiente. Si la ejecutamos sucesivas veces se puede observar que el valor de $f$ se reduce:

In [33]:
def iteration_gradient_descent(learning_rate = 0.01, show_value=False):
    value = f(w)
    if show_value:
        print(f'f({w.item()}) = {value.item()}')
    value.backward()
    w.data = w.data - learning_rate * w.grad
    w.grad.zero_()

iteration_gradient_descent(show_value=True)

f(0.9700387716293335) = 14.120742797851562


Ejecutando dentro de un loop para hacer muchas iteraciones:

In [34]:
for iteration_number in range(1000):
    iteration_gradient_descent(show_value = (iteration_number%100 == 0))

f(1.0106379985809326) = 13.957561492919922
f(2.7361714839935303) = 10.069605827331543
f(2.96501088142395) = 10.001224517822266
f(2.995359420776367) = 10.000021934509277
f(2.999384880065918) = 10.0
f(2.9999184608459473) = 10.0
f(2.999988317489624) = 10.0
f(2.9999942779541016) = 10.0
f(2.9999942779541016) = 10.0
f(2.9999942779541016) = 10.0


Finalmente hemos encontrado el valor de $x$ que minimiza $f(x)$

In [35]:
f(w.data), w.data

(tensor([10.]), tensor([3.0000]))

### Otra aplicación de descenso por gradiente

Vimos que descenso por gradiente sirve para encontrar el mínimo de funciones. Pero también es útil en problemas que queremos encontrar cierta incógnita. Veremos cómo se puede hallar la inversa de una matriz con descenso por gradiente.

Recordar que si tenemos una matriz $A$:
\begin{bmatrix}
    1 & 2 & 3 \\
    0 & 1 & 4 \\
    5 & 6 & 0
\end{bmatrix}
Su inversa $A^{-1}$ es:
\begin{bmatrix}
    -24 & 18 & 5 \\
    20 & -15 & -4 \\
    -5 & 4 & 1
\end{bmatrix}
Debido a que el [producto](https://en.wikipedia.org/wiki/Matrix_multiplication) de ambas es la identidad $A \times A^{-1} = I$. 

Recordar que la identidad $I$ es:
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1
\end{bmatrix}

In [36]:
values = [[1,2,3],
          [0,1,4],
          [5,6,0]]
A = torch.tensor(values, dtype=torch.float)
A

tensor([[1., 2., 3.],
        [0., 1., 4.],
        [5., 6., 0.]])

Existen métodos para calcular la inversa de una matrix. Pytorch ya posee dicha funcionalidad como una función de librería:

In [37]:
A_inversed = torch.inverse(A)
A_inversed

tensor([[-24.0000,  18.0000,   5.0000],
        [ 20.0000, -15.0000,  -4.0000],
        [ -5.0000,   4.0000,   1.0000]])

Pytorch posee además métodos para definir matrices útiles como la identidad:

In [38]:
torch.eye(A.shape[0])

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

Podemos corroborar que el producto de la inversa calculada por la matriz original es la identidad. Si se mira con atención en la diagonal hay todos $1$ y en las entradas que no pertenecen a la diagonal hay números muy pequeños (casi 0). Esto es producto a [errores numéricos](https://en.wikipedia.org/wiki/Numerical_error), (recordar que la representación como float no es exacta)

In [39]:
A_inversed @ A

tensor([[ 1.0000e+00,  3.8147e-06,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00, -3.8147e-06],
        [ 4.7684e-07,  1.9073e-06,  1.0000e+00]])

Pero cómo usamos descenso por gradiente para encontrar $A^{-1}$? No sirve sólo para problemas de minimización?

$A^{-1}$ es la única matriz que multiplicada por $A$ da la identidad:

$A \times A^{-1} = I$

Lo que es equivalente a decir:

$A \times A^{-1} - I = 0$

Y a su vez, es equivalente a:

$||A \times A^{-1} - I ||_2 = 0$

La norma $|| . ||_2$ es siempre positiva (o cero). Por lo cual, encontrar $A^{-1}$ es equivalente a minimizar:

$f(W) = ||A \times W - I ||_2 $

Comenzaremos con una matriz $W_0$ aleatoria e iremos actualizando sus entradas con descenso por gradiente para minimizar $f$ definida tal como se muestra arriba.

In [40]:
def f(W):
    dim = A.shape[0]
    I = torch.eye(dim)
    return torch.norm(A @ W - I)

Nuestro $W$ inicial está lejos de ser la inversa $A_{-1}$ buscada:

In [41]:
W = torch.rand((3,3), requires_grad=True)
W

tensor([[0.2712, 0.8705, 0.3353],
        [0.1286, 0.1051, 0.8353],
        [0.2796, 0.7981, 0.5629]], requires_grad=True)

$f(W)$ que puede ser interpretado como "el error" de nuestra solución está lejos de ser $0$:

In [42]:
f(W)

tensor(10.1950, grad_fn=<LinalgVectorNormBackward0>)

La función que realiza una iteración de descenso por gradiente es igual que la que usamos para nuestra función cuadrática.
También podemos corrobarar, una vez más que ejecutando sucesivas iteraciones el valor de $f$ es cada vez menor:

In [43]:
def iteration_gradient_descent(learning_rate = 0.01, show_value=False):
    value = f(W)
    if show_value:
        print(f'f(W) = {value.item()}')
        print(W)
    value.backward()
    W.data = W.data - learning_rate * W.grad
    W.grad.zero_()

iteration_gradient_descent(show_value=True)

f(W) = 10.194985389709473
tensor([[0.2712, 0.8705, 0.3353],
        [0.1286, 0.1051, 0.8353],
        [0.2796, 0.7981, 0.5629]], requires_grad=True)


Para hacer varias iteraciones utilizaremos learning rates cada vez más chicos (recordar que al aproximarse al mínimo es conveniente reducir el learning rate para no "saltar" por encima del mínimo):

In [44]:
for lr_exponent in range(1,7):
    n_iter_to_show = 100000
    n_iter = n_iter_to_show * 3
    learning_rate = 0.1 ** lr_exponent
    for iteration_number in range(n_iter):
        iteration_gradient_descent(show_value = (iteration_number%n_iter_to_show == 0), learning_rate = learning_rate)

f(W) = 9.554241180419922
tensor([[0.2604, 0.8426, 0.3037],
        [0.1142, 0.0667, 0.7916],
        [0.2736, 0.7789, 0.5400]], requires_grad=True)
f(W) = 3.4269797801971436
tensor([[-20.1295,  14.9954,   3.9967],
        [ 16.6526, -12.8340,  -3.5485],
        [ -4.1847,   3.3359,   0.7587]], requires_grad=True)
f(W) = 3.4276392459869385
tensor([[-23.4183,  17.3784,   4.6853],
        [ 19.3907, -14.8181,  -4.1224],
        [ -4.8851,   3.8434,   0.9053]], requires_grad=True)
f(W) = 3.424090623855591
tensor([[-23.9549,  17.7671,   4.7977],
        [ 19.8374, -15.1414,  -4.2155],
        [ -4.9994,   3.9262,   0.9293]], requires_grad=True)
f(W) = 0.3360155522823334
tensor([[-23.9694,  17.9594,   4.9746],
        [ 19.9623, -14.9990,  -4.0166],
        [ -4.9944,   3.9890,   0.9919]], requires_grad=True)
f(W) = 0.3360155522823334
tensor([[-23.9694,  17.9594,   4.9746],
        [ 19.9623, -14.9990,  -4.0166],
        [ -4.9944,   3.9890,   0.9919]], requires_grad=True)
f(W) = 0.336015552

El $W$ obtenido es una aproximacion relativamente buena de $A^{-1}$

In [45]:
W

tensor([[-23.9648,  17.9756,   4.9926],
        [ 19.9707, -14.9797,  -3.9938],
        [ -4.9925,   3.9948,   0.9984]], requires_grad=True)

In [46]:
A_inversed

tensor([[-24.0000,  18.0000,   5.0000],
        [ 20.0000, -15.0000,  -4.0000],
        [ -5.0000,   4.0000,   1.0000]])

#### Ejercicio 3: "Resolviendo un sistema de ecuaciones utilizando descenso por gradiente" (opcional)

Utilizar el método de descenso por gradiente para resolver el siguiente sistema de ecuaciones:

$ 3 x + 4 y - 2 z = 0$

$ 2 x - 3 y + 4 z = 11$

$ x - 2 y + 3 z = 7$

Hint: recordar la representación matricial del sistema de ecuaciones.